# Introduction

### Scenario

A couple originally from France living and working in a Small Dutch city called Wageningen. However, at some point, they were thought to move back in there a home city in France, Lyon. Wageningen is a relatively a small city in comparison to Lyon. Most of the amenities such as library, hospital, markets, are located near to the city center. On the other hand, Lyon has huge infrastructure and sometimes it is hard to find a good neighborhood that satisfy all needs of old age people. For example, this french couple is looking forward to having the following best facilities in their nearby neighborhood

* Local library
* Health centre
* Grocery store 
* Social centre
* Performing Arts Venue

In order to meet their requirement these couple is looking for a company that can assist in their relocation.  

### Business case 

A fictitious company called RELOCATE ME facilitate retired people to relocate in their preferred city and in return company earn revenue. The company proposes the best relocation package considering the specific requirement such as social engagement, partial jobs, medical visits, etc., for retired people. Often, finding such recommendations are not easy and require a detailed analysis of the particular city and neighborhood. 

### Role of data science

RELOCATE ME uses the power of data science to explore cities, taking into account the various needs of retired or old age people. RELOCATE ME uses data from different public sources to find, the better the recommendations for their client.

### Data used for problem solving 

The following data sets are used to in this case study:
* Foursquare data to find and compare locations
* Public data set about the city Lyon in France 
  https://data.beta.grandlyon.com/en/accueil

# Methodology 

In order to facilitate their client RELOCATE ME uses the exploratory analysis of the current and requested city of relocation. Moreover, it follows the following methodology to advise better location for elderly people. 

- Find the specific requirements such as the Health center, Social center, etc. from the clients. 

- Further is analyze their current city of living to understand available services and facilities 

- Based on previous analysis, company start analyzing their requested city based on their demand

- Further, the company perform categorial analysis from the Foursquare database

- The city divided into the different cluster and each cluster is analyzed taking into account different category.

- Finally, the company advise the best district considering clients requirements

### City Wageningen in Netherlands

Following analysis represents the vizulization of Wageningen City focusing on following details 

* Local library
* Health centre
* Grocery store 
* Social centre
* Performing Arts Venue

In [1]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [95]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [103]:

address = 'Wageningen'

geolocator = Nominatim(user_agent="Wageningen_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('31 Bowlespark, Wageningen are {}, {}.'.format(latitude, longitude))

31 Bowlespark, Wageningen are 51.9685817, 5.66829806182796.


In [105]:
CLIENT_ID = 'X52HWHL32ARKRE4ZGHER5IMTAMCCMT2SBXVYCYE2DSVQ1Z5U' # your Foursquare ID
CLIENT_SECRET = '15R0HBHZ1TWTCB1EDA4BNTPMLDLBRVVO0FKGLP0DBGSPDPTK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X52HWHL32ARKRE4ZGHER5IMTAMCCMT2SBXVYCYE2DSVQ1Z5U
CLIENT_SECRET:15R0HBHZ1TWTCB1EDA4BNTPMLDLBRVVO0FKGLP0DBGSPDPTK


In [106]:
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
#latitude=51.9649102
#longitude=5.6674073

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?categoryId=4bf58dd8d48988d12f941735,52e81612bcbc57f1066b7a33,4bf58dd8d48988d196941735,4bf58dd8d48988d118951735,58daa1558bbb0b01f18ec1f7,5744ccdfe4b0c0459246b4af,56aa371be4b08b9a8d573526,4bf58dd8d48988d104941735,4bf58dd8d48988d1f2931735 &client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?categoryId=4bf58dd8d48988d12f941735,52e81612bcbc57f1066b7a33,4bf58dd8d48988d196941735,4bf58dd8d48988d118951735,58daa1558bbb0b01f18ec1f7,5744ccdfe4b0c0459246b4af,56aa371be4b08b9a8d573526,4bf58dd8d48988d104941735,4bf58dd8d48988d1f2931735 &client_id=X52HWHL32ARKRE4ZGHER5IMTAMCCMT2SBXVYCYE2DSVQ1Z5U&client_secret=15R0HBHZ1TWTCB1EDA4BNTPMLDLBRVVO0FKGLP0DBGSPDPTK&v=20180605&ll=51.9685817,5.66829806182796&radius=1000&limit=200'

In [107]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d53d3aa6bdee6002c1e64fc'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Wageningen',
  'headerFullLocation': 'Wageningen',
  'headerLocationGranularity': 'city',
  'query': 'library',
  'totalResults': 17,
  'suggestedBounds': {'ne': {'lat': 51.97758170900001,
    'lng': 5.682878973799916},
   'sw': {'lat': 51.95958169099999, 'lng': 5.6537171498560035}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b9a89eff964a5201abf35e3',
       'name': 'Bblthk',
       'location': {'address': 'Schoolstraat 2',
        'crossStreet': 'Bergstraat',
        'lat': 51.9676303972737,
        'lng': 5.66558083499671,
        'labeledLatLngs': [{'label': 'di

In [108]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [109]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(100)

,name,categories,lat,lng
0,Bblthk,Library,51.967630,5.665581
1,Medisch Centrum De Poort,Medical Center,51.968372,5.665033
2,Toko Indrani,Grocery Store,51.967019,5.663356
3,Egyptian Foodies,Grocery Store,51.965870,5.660700
4,ALDI,Supermarket,51.973753,5.673201
5,"Verpleeghuis Rumah Kita, Zinzia Zorggroep",Medical Center,51.970776,5.657255
6,Vilente Costerweg,Hospital,51.967107,5.656565
7,M-Visio,Medical Center,51.967362,5.655505
8,Theater Junushoff,Theater,51.966883,5.661508
9,Fysiotherapie Ingrid Marcusse Wageningen,Physical Therapist,51.975620,5.659310


In [110]:

map_sg = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg

# Conclusion:

Wageningen is a very small town, so far Foursquare query return only a few venues. Most of the venues are easily assessable by an elderly couple. In this way, the company understood the currently available facilities in the vicinity of the client.

# Data preparition for city Lyon

Lyon is relatively big city and has five district (Arrondissement). A Wikipedia page (https://fr.wikipedia.org/wiki/Lyon) provides succinct details about the different districts in Lyon. Further, a set of data is obtained from a public website about Lyon (https://data.beta.grandlyon.com/en/accueil)and a separate Lyon.csv file is prepared to find post code of each district.  Based on, geographical coordinated the following map is obtained. 



In [56]:
pd.set_option('display.width', 1000)
df = pd.read_html('https://fr.wikipedia.org/wiki/Lyon') ###<-- List
df_Lyon=df[8]
df_Lyon.columns = ['Arrondissement1', 'quartier1', 'Arrondissement2', 'quartier2',]
df_Lyon.drop(df_Lyon.index[[0]],inplace=True)
DF1=df_Lyon['Arrondissement1'].append(df_Lyon['Arrondissement2']).reset_index(drop=True)
DF2=df_Lyon['quartier1'].append(df_Lyon['quartier2']).reset_index(drop=True)

df_main = pd.concat([DF1, DF2], axis=1)
df_main = df_main.set_axis(['Arrondissement', 'quartier'], axis=1, inplace=False)
df_main.dropna(inplace=True)
df_main

,Arrondissement,quartier
0,1er arrondissement,"Les Terreaux, les pentes de la Croix-Rousse, q..."
1,3e arrondissement,"Moncey, la Part-Dieu, la Villette, Montchat, l..."
2,5e arrondissement,"le Vieux Lyon (Saint-Jean, Saint-Paul, Saint-G..."
3,7e arrondissement,"La Mouche, la Guillotière-Sud, Jean Macé, Gerl..."
4,9e arrondissement,"Vaise, Industrie, Champvert-Nord, Gorge de Lou..."
5,2e arrondissement,"Les Cordeliers, les Jacobins, la Presqu'île, A..."
6,4e arrondissement,"La Croix-Rousse, Serin-Saint-Charles, le Clos ..."
7,6e arrondissement,"Les Brotteaux, Bellecombe, la Tête d'or, Massé..."
8,8e arrondissement,"Monplaisir, le Bachut, Mermoz, Les États-Unis,..."


In [57]:
df = pd.read_html('https://fr.wikipedia.org/wiki/Arrondissements_de_Lyon') ###<-- List
df_Lyon=df[0]
#df_Toronto.columns.values[2] = "Neighborhood"
#rows=['Code Insee', 'Date de création','Population (2015)[2]','Population (2006)','Population (1990)','Population (1982)','Population (1975)','Population (1968)','Superficie (ha)','Densité (hab./km²) (2015)']
df_Lyon_district=df_Lyon.drop(['Code Insee','Date de création','Population (2015)[2]','Population (2011)','Population (2006)','Population (1999)','Population (1990)','Population (1982)','Population (1975)','Population (1968)','Superficie (ha)','Densité (hab./km²) (2015)'], axis=1)
df_Lyon_district

,Code Postal,Arrondissement
0,69001,1er arrondissement
1,69002,2e arrondissement
2,69003,3e arrondissement
3,69004,4e arrondissement
4,69005,5e arrondissement
5,69006,6e arrondissement
6,69007,7e arrondissement
7,69008,8e arrondissement
8,69009,9e arrondissement


In [198]:

pd_concated_data=pd.concat
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.


body = client_3d64a4e2f4df4fc78d4465e3fda7822f.get_object(Bucket='capstoneprojectweek2-donotdelete-pr-dkn96chcmrakch',Key='Lyon.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_geo = pd.read_csv(body)
df_geo.head()



,Postal Code,Latitude,Longitude
0,69001,45.767699,4.835490
1,69002,45.751112,4.827374
2,69003,45.759934,4.849390
3,69004,45.778046,4.825031
4,69005,45.763572,4.828436


In [59]:

#([df_Lyon_district,df_geo],join='outer', axis=1)
pd_concated_data = df_Lyon_district.join(df_geo.set_index('Postal Code'), on='Code Postal')
pd_concated_data = pd_concated_data.join(df_main.set_index('Arrondissement'), on='Arrondissement')
pd_concated_data_new=pd_concated_data.set_axis(['Postcode', 'Borough','Latitude','Longitude','Neighborhood'], axis=1, inplace=False)
pd_concated_data_new


,Postcode,Borough,Latitude,Longitude,Neighborhood
0,69001,1er arrondissement,45.767699,4.835490,"Les Terreaux, les pentes de la Croix-Rousse, q..."
1,69002,2e arrondissement,45.751112,4.827374,"Les Cordeliers, les Jacobins, la Presqu'île, A..."
2,69003,3e arrondissement,45.759934,4.849390,"Moncey, la Part-Dieu, la Villette, Montchat, l..."
3,69004,4e arrondissement,45.778046,4.825031,"La Croix-Rousse, Serin-Saint-Charles, le Clos ..."
4,69005,5e arrondissement,45.763572,4.828436,"le Vieux Lyon (Saint-Jean, Saint-Paul, Saint-G..."
5,69006,6e arrondissement,45.746821,4.841941,"Les Brotteaux, Bellecombe, la Tête d'or, Massé..."
6,69007,7e arrondissement,45.751112,4.827374,"La Mouche, la Guillotière-Sud, Jean Macé, Gerl..."
7,69008,8e arrondissement,48.864434,2.321661,"Monplaisir, le Bachut, Mermoz, Les États-Unis,..."
8,69009,9e arrondissement,45.774000,4.806249,"Vaise, Industrie, Champvert-Nord, Gorge de Lou..."


In [60]:
map_Lyon = folium.Map(location=[45.767699, 4.835490], zoom_start=11)

pd_concated_data_new.apply(lambda row:folium.CircleMarker(location=[row["Latitude"], row["Longitude"]], 
                                              radius=10, fill_color='#3186cc',fill=True,fill_opacity=0.7)
                                             .add_to(map_Lyon), axis=1)

map_Lyon

In [61]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(pd_concated_data_new['Borough'].unique()),
        pd_concated_data_new.shape[0]
    )
)

The dataframe has 9 boroughs and 9 neighborhoods.


In [181]:
address = 'Lyon'

geolocator = Nominatim(user_agent="Lyon_explorer")
location = geolocator.geocode(address)
neighborhood_latitude = location.latitude
neighborhood_longitude = location.longitude
print('The geograpical coordinate of Lyon City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lyon City are 51.9685817, 5.66829806182796.


In [197]:
# create URL
radius = 5000
LIMIT = 500
neighborhood_latitude= 45.767699
neighborhood_longitude= 4.835490




url = 'https://api.foursquare.com/v2/venues/explore?&categoryId=4bf58dd8d48988d12f941735,52e81612bcbc57f1066b7a33,4bf58dd8d48988d196941735,4bf58dd8d48988d118951735,58daa1558bbb0b01f18ec1f7,5744ccdfe4b0c0459246b4af,56aa371be4b08b9a8d573526,4bf58dd8d48988d104941735,4bf58dd8d48988d1f2931735 &client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    'X52HWHL32ARKRE4ZGHER5IMTAMCCMT2SBXVYCYE2DSVQ1Z5U', 
    '15R0HBHZ1TWTCB1EDA4BNTPMLDLBRVVO0FKGLP0DBGSPDPTK',
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


url 

'https://api.foursquare.com/v2/venues/explore?&categoryId=4bf58dd8d48988d12f941735,52e81612bcbc57f1066b7a33,4bf58dd8d48988d196941735,4bf58dd8d48988d118951735,58daa1558bbb0b01f18ec1f7,5744ccdfe4b0c0459246b4af,56aa371be4b08b9a8d573526,4bf58dd8d48988d104941735,4bf58dd8d48988d1f2931735 &client_id=X52HWHL32ARKRE4ZGHER5IMTAMCCMT2SBXVYCYE2DSVQ1Z5U&client_secret=15R0HBHZ1TWTCB1EDA4BNTPMLDLBRVVO0FKGLP0DBGSPDPTK&v=20180605&ll=45.767699,4.83549&radius=5000&limit=500'

In [183]:
results = requests.get(url).json()
results
#results = requests.get(url).json()["response"]["venues"]


#from pandas.io.json import json_normalize
#df = pd.DataFrame.from_dict(json_normalize(results), orient='columns')
#print (df)

#venues = results['response']['groups'][0]['item']
#print(venues)
#results['response']['groups'][0]['items']

{'meta': {'code': 200, 'requestId': '5d541274ba921100250e5727'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Lyon',
  'headerFullLocation': 'Lyon',
  'headerLocationGranularity': 'city',
  'query': 'library',
  'totalResults': 82,
  'suggestedBounds': {'ne': {'lat': 45.812699045000045,
    'lng': 4.899879350266952},
   'sw': {'lat': 45.722698954999956, 'lng': 4.771100649733048}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bf6bedc5efe2d7ffbac6734',
       'name': 'Simply Market',
       'location': {'address': '38 rue Garibaldi',
        'lat': 45.77104030244457,
        'lng': 4.850027738363078,
        'labeledLatLngs': [{'label': 'display',
          'lat': 45.771040302

In [184]:
def get_category_type(row):
    try:
        categories_list = row['categories']
        
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [187]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Simply Market,Grocery Store,45.771040,4.850028
1,Marché Presqu'île,Grocery Store,45.766142,4.831958
2,Monoprix,Department Store,45.763414,4.851751
3,Monop'daily,Grocery Store,45.760523,4.859357
4,Super U,Grocery Store,45.760131,4.837677


In [188]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

82 venues were returned by Foursquare.


In [189]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
         # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&categoryId=4bf58dd8d48988d12f941735,4bf58dd8d48988d196941735,52e81612bcbc57f1066b7a33,4bf58dd8d48988d118951735,58daa1558bbb0b01f18ec1f7,5744ccdfe4b0c0459246b4af,56aa371be4b08b9a8d573526,4bf58dd8d48988d104941735,4bf58dd8d48988d1f2931735 &client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT) 
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [190]:
Lyon_district_venues = getNearbyVenues(names=pd_concated_data_new['Borough'],
                                   latitudes=pd_concated_data_new['Latitude'],
                                   longitudes=pd_concated_data_new['Longitude']
                                  )



1er arrondissement
2e arrondissement
3e arrondissement
4e arrondissement
5e arrondissement
6e arrondissement
7e arrondissement
8e arrondissement
9e arrondissement


In [191]:
print(Lyon_district_venues.shape)
Lyon_district_venues

(421, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,1er arrondissement,45.767699,4.835490,Marché Presqu'île,45.766142,4.831958,Grocery Store
1,1er arrondissement,45.767699,4.835490,Simply Market,45.771040,4.850028,Grocery Store
2,1er arrondissement,45.767699,4.835490,Super U,45.760131,4.837677,Grocery Store
3,1er arrondissement,45.767699,4.835490,Marché U,45.759873,4.837600,Grocery Store
4,1er arrondissement,45.767699,4.835490,Petit Casino,45.759717,4.834878,Grocery Store
5,1er arrondissement,45.767699,4.835490,Laboratoire d'analyses medicales Duquesne,45.772674,4.845373,Medical Center
6,1er arrondissement,45.767699,4.835490,Carrefour City Lyon Croix Rousse,45.776208,4.833073,Grocery Store
7,1er arrondissement,45.767699,4.835490,Bibliotheque Municipale De Lyon,45.775946,4.830867,Library
8,1er arrondissement,45.767699,4.835490,Hôtel-Dieu,45.758863,4.836624,Hospital
9,1er arrondissement,45.767699,4.835490,Monoprix,45.763414,4.851751,Department Store


In [192]:
Lyon_district_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
1er arrondissement,46,46,46,46,46,46
2e arrondissement,32,32,32,32,32,32
3e arrondissement,43,43,43,43,43,43
4e arrondissement,52,52,52,52,52,52
5e arrondissement,45,45,45,45,45,45
6e arrondissement,38,38,38,38,38,38
7e arrondissement,32,32,32,32,32,32
8e arrondissement,100,100,100,100,100,100
9e arrondissement,33,33,33,33,33,33


In [193]:
print('There are {} uniques categories.'.format(len(Lyon_district_venues['Venue Category'].unique())))

There are 38 uniques categories.


In [194]:
# one hot encoding
Lyon_district_onehot = pd.get_dummies(Lyon_district_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Lyon_district_onehot['Neighborhood'] = Lyon_district_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Lyon_district_onehot.columns[-1]] + list(Lyon_district_onehot.columns[:-1])
Lyon_district_onehot = Lyon_district_onehot[fixed_columns]

Lyon_district_onehot.head()

,Neighborhood,Alternative Healer,Art Gallery,Bistro,Bookstore,Business Service,Café,Chiropractor,College Library,Comedy Club,Convenience Store,Dance Studio,Department Store,Discount Store,Doctor's Office,Food & Drink Shop,General College & University,Gourmet Shop,Grocery Store,Home Service,Hospital,Japanese Restaurant,Library,Massage Studio,Medical Center,Medical Lab,Medical Supply Store,Miscellaneous Shop,Nightclub,Non-Profit,Opera House,Optical Shop,Performing Arts Venue,School,Shopping Mall,Social Club,Supermarket,Theater,Wine Shop
0,1er arrondissement,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1er arrondissement,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1er arrondissement,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1er arrondissement,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1er arrondissement,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [195]:
Lyon_district_grouped = Lyon_district_onehot.groupby('Neighborhood').mean().reset_index()
Lyon_district_grouped

,Neighborhood,Alternative Healer,Art Gallery,Bistro,Bookstore,Business Service,Café,Chiropractor,College Library,Comedy Club,Convenience Store,Dance Studio,Department Store,Discount Store,Doctor's Office,Food & Drink Shop,General College & University,Gourmet Shop,Grocery Store,Home Service,Hospital,Japanese Restaurant,Library,Massage Studio,Medical Center,Medical Lab,Medical Supply Store,Miscellaneous Shop,Nightclub,Non-Profit,Opera House,Optical Shop,Performing Arts Venue,School,Shopping Mall,Social Club,Supermarket,Theater,Wine Shop
0,1er arrondissement,0.021739,0.00,0.00,0.00,0.000000,0.00,0.021739,0.00,0.021739,0.00,0.021739,0.021739,0.000000,0.065217,0.021739,0.00,0.00,0.282609,0.000000,0.108696,0.00,0.021739,0.000000,0.152174,0.043478,0.043478,0.000000,0.00,0.000000,0.021739,0.000000,0.00,0.021739,0.000000,0.00,0.065217,0.043478,0.00
1,2e arrondissement,0.062500,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.031250,0.00,0.000000,0.000000,0.031250,0.062500,0.000000,0.00,0.00,0.343750,0.000000,0.062500,0.00,0.062500,0.000000,0.125000,0.062500,0.000000,0.000000,0.00,0.000000,0.031250,0.000000,0.00,0.000000,0.000000,0.00,0.093750,0.031250,0.00
2,3e arrondissement,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.023256,0.00,0.000000,0.023256,0.023256,0.069767,0.046512,0.00,0.00,0.372093,0.000000,0.069767,0.00,0.023256,0.000000,0.139535,0.046512,0.023256,0.000000,0.00,0.023256,0.023256,0.000000,0.00,0.000000,0.000000,0.00,0.069767,0.023256,0.00
3,4e arrondissement,0.076923,0.00,0.00,0.00,0.019231,0.00,0.019231,0.00,0.000000,0.00,0.019231,0.000000,0.000000,0.153846,0.000000,0.00,0.00,0.115385,0.038462,0.096154,0.00,0.057692,0.019231,0.153846,0.038462,0.019231,0.000000,0.00,0.000000,0.019231,0.000000,0.00,0.019231,0.019231,0.00,0.038462,0.076923,0.00
4,5e arrondissement,0.044444,0.00,0.00,0.00,0.000000,0.00,0.022222,0.00,0.022222,0.00,0.022222,0.022222,0.000000,0.044444,0.000000,0.00,0.00,0.266667,0.000000,0.088889,0.00,0.044444,0.022222,0.133333,0.066667,0.022222,0.000000,0.00,0.000000,0.022222,0.000000,0.00,0.022222,0.000000,0.00,0.088889,0.044444,0.00
5,6e arrondissement,0.052632,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.026316,0.00,0.000000,0.026316,0.026316,0.026316,0.000000,0.00,0.00,0.289474,0.000000,0.052632,0.00,0.052632,0.000000,0.210526,0.052632,0.000000,0.026316,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.131579,0.026316,0.00
6,7e arrondissement,0.062500,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.031250,0.00,0.000000,0.000000,0.031250,0.062500,0.000000,0.00,0.00,0.343750,0.000000,0.062500,0.00,0.062500,0.000000,0.125000,0.062500,0.000000,0.000000,0.00,0.000000,0.031250,0.000000,0.00,0.000000,0.000000,0.00,0.093750,0.031250,0.00
7,8e arrondissement,0.000000,0.01,0.01,0.02,0.000000,0.02,0.000000,0.02,0.000000,0.04,0.010000,0.020000,0.000000,0.020000,0.000000,0.01,0.02,0.220000,0.000000,0.040000,0.01,0.120000,0.000000,0.080000,0.010000,0.000000,0.000000,0.01,0.000000,0.010000,0.000000,0.02,0.000000,0.000000,0.01,0.000000,0.260000,0.01
8,9e arrondissement,0.060606,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.181818,0.000000,0.00,0.00,0.030303,0.060606,0.090909,0.00,0.090909,0.030303,0.181818,0.030303,0.030303,0.000000,0.00,0.000000,0.000000,0.030303,0.00,0.030303,0.030303,0.00,0.090909,0.030303,0.00


In [196]:
num_top_venues = 15

for hood in Lyon_district_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Lyon_district_grouped[Lyon_district_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----1er arrondissement----
                   venue  freq
0          Grocery Store  0.28
1         Medical Center  0.15
2               Hospital  0.11
3        Doctor's Office  0.07
4            Supermarket  0.07
5                Theater  0.04
6   Medical Supply Store  0.04
7            Medical Lab  0.04
8                 School  0.02
9            Opera House  0.02
10               Library  0.02
11     Food & Drink Shop  0.02
12    Alternative Healer  0.02
13          Chiropractor  0.02
14          Dance Studio  0.02


----2e arrondissement----
                 venue  freq
0        Grocery Store  0.34
1       Medical Center  0.12
2          Supermarket  0.09
3   Alternative Healer  0.06
4      Doctor's Office  0.06
5          Medical Lab  0.06
6              Library  0.06
7             Hospital  0.06
8          Comedy Club  0.03
9              Theater  0.03
10      Discount Store  0.03
11         Opera House  0.03
12           Bookstore  0.00
13              Bistro  0.00
14         Soc

In [150]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [165]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Lyon_district_grouped['Neighborhood']

for ind in np.arange(Lyon_district_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Lyon_district_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1er arrondissement,Grocery Store,Medical Center,Hospital,Doctor's Office,Supermarket,Medical Lab,Theater,Medical Supply Store,Opera House,Comedy Club
1,2e arrondissement,Grocery Store,Medical Center,Supermarket,Alternative Healer,Medical Lab,Hospital,Library,Doctor's Office,Opera House,Theater
2,3e arrondissement,Grocery Store,Medical Center,Doctor's Office,Supermarket,Hospital,Food & Drink Shop,Medical Lab,Discount Store,Department Store,Theater
3,4e arrondissement,Doctor's Office,Medical Center,Grocery Store,Hospital,Alternative Healer,Theater,Library,Medical Lab,Home Service,Supermarket
4,5e arrondissement,Grocery Store,Medical Center,Supermarket,Hospital,Medical Lab,Alternative Healer,Theater,Library,Doctor's Office,Chiropractor
5,6e arrondissement,Grocery Store,Medical Center,Supermarket,Alternative Healer,Medical Lab,Hospital,Library,Discount Store,Theater,Department Store
6,7e arrondissement,Grocery Store,Medical Center,Supermarket,Alternative Healer,Medical Lab,Hospital,Library,Doctor's Office,Opera House,Theater
7,8e arrondissement,Theater,Grocery Store,Library,Medical Center,Convenience Store,Hospital,Doctor's Office,Bookstore,Performing Arts Venue,Gourmet Shop
8,9e arrondissement,Medical Center,Doctor's Office,Supermarket,Hospital,Library,Home Service,Alternative Healer,Optical Shop,Medical Lab,Grocery Store


In [166]:
kclusters = 7

Lyon_district_grouped_clustering = Lyon_district_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(Lyon_district_grouped_clustering)
kmeans.labels_[0:10]’

array([4, 3, 6, 5, 4, 0, 3, 2, 1], dtype=int32)

In [167]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Lyon_district_merged = pd_concated_data_new


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Lyon_district_merged = Lyon_district_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Borough')

Lyon_district_merged 

,Postcode,Borough,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,69001,1er arrondissement,45.767699,4.835490,"Les Terreaux, les pentes de la Croix-Rousse, q...",4,Grocery Store,Medical Center,Hospital,Doctor's Office,Supermarket,Medical Lab,Theater,Medical Supply Store,Opera House,Comedy Club
1,69002,2e arrondissement,45.751112,4.827374,"Les Cordeliers, les Jacobins, la Presqu'île, A...",3,Grocery Store,Medical Center,Supermarket,Alternative Healer,Medical Lab,Hospital,Library,Doctor's Office,Opera House,Theater
2,69003,3e arrondissement,45.759934,4.849390,"Moncey, la Part-Dieu, la Villette, Montchat, l...",6,Grocery Store,Medical Center,Doctor's Office,Supermarket,Hospital,Food & Drink Shop,Medical Lab,Discount Store,Department Store,Theater
3,69004,4e arrondissement,45.778046,4.825031,"La Croix-Rousse, Serin-Saint-Charles, le Clos ...",5,Doctor's Office,Medical Center,Grocery Store,Hospital,Alternative Healer,Theater,Library,Medical Lab,Home Service,Supermarket
4,69005,5e arrondissement,45.763572,4.828436,"le Vieux Lyon (Saint-Jean, Saint-Paul, Saint-G...",4,Grocery Store,Medical Center,Supermarket,Hospital,Medical Lab,Alternative Healer,Theater,Library,Doctor's Office,Chiropractor
5,69006,6e arrondissement,45.746821,4.841941,"Les Brotteaux, Bellecombe, la Tête d'or, Massé...",0,Grocery Store,Medical Center,Supermarket,Alternative Healer,Medical Lab,Hospital,Library,Discount Store,Theater,Department Store
6,69007,7e arrondissement,45.751112,4.827374,"La Mouche, la Guillotière-Sud, Jean Macé, Gerl...",3,Grocery Store,Medical Center,Supermarket,Alternative Healer,Medical Lab,Hospital,Library,Doctor's Office,Opera House,Theater
7,69008,8e arrondissement,48.864434,2.321661,"Monplaisir, le Bachut, Mermoz, Les États-Unis,...",2,Theater,Grocery Store,Library,Medical Center,Convenience Store,Hospital,Doctor's Office,Bookstore,Performing Arts Venue,Gourmet Shop
8,69009,9e arrondissement,45.774000,4.806249,"Vaise, Industrie, Champvert-Nord, Gorge de Lou...",1,Medical Center,Doctor's Office,Supermarket,Hospital,Library,Home Service,Alternative Healer,Optical Shop,Medical Lab,Grocery Store


In [168]:
Lyon_district_merged.loc[Lyon_district_merged['Cluster Labels'] == 0, Lyon_district_merged.columns[[1] + list(range(5, Lyon_district_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,6e arrondissement,0,Grocery Store,Medical Center,Supermarket,Alternative Healer,Medical Lab,Hospital,Library,Discount Store,Theater,Department Store


In [169]:
Lyon_district_merged.loc[Lyon_district_merged['Cluster Labels'] == 1, Lyon_district_merged.columns[[1] + list(range(5, Lyon_district_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,9e arrondissement,1,Medical Center,Doctor's Office,Supermarket,Hospital,Library,Home Service,Alternative Healer,Optical Shop,Medical Lab,Grocery Store


In [170]:
Lyon_district_merged.loc[Lyon_district_merged['Cluster Labels'] == 2, Lyon_district_merged.columns[[1] + list(range(5, Lyon_district_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,8e arrondissement,2,Theater,Grocery Store,Library,Medical Center,Convenience Store,Hospital,Doctor's Office,Bookstore,Performing Arts Venue,Gourmet Shop


In [171]:
Lyon_district_merged.loc[Lyon_district_merged['Cluster Labels'] == 3, Lyon_district_merged.columns[[1] + list(range(5, Lyon_district_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,2e arrondissement,3,Grocery Store,Medical Center,Supermarket,Alternative Healer,Medical Lab,Hospital,Library,Doctor's Office,Opera House,Theater
6,7e arrondissement,3,Grocery Store,Medical Center,Supermarket,Alternative Healer,Medical Lab,Hospital,Library,Doctor's Office,Opera House,Theater


In [172]:
Lyon_district_merged.loc[Lyon_district_merged['Cluster Labels'] == 4, Lyon_district_merged.columns[[1] + list(range(5, Lyon_district_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1er arrondissement,4,Grocery Store,Medical Center,Hospital,Doctor's Office,Supermarket,Medical Lab,Theater,Medical Supply Store,Opera House,Comedy Club
4,5e arrondissement,4,Grocery Store,Medical Center,Supermarket,Hospital,Medical Lab,Alternative Healer,Theater,Library,Doctor's Office,Chiropractor


In [173]:
Lyon_district_merged.loc[Lyon_district_merged['Cluster Labels'] == 5, Lyon_district_merged.columns[[1] + list(range(5, Lyon_district_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,4e arrondissement,5,Doctor's Office,Medical Center,Grocery Store,Hospital,Alternative Healer,Theater,Library,Medical Lab,Home Service,Supermarket


In [174]:
Lyon_district_merged.loc[Lyon_district_merged['Cluster Labels'] == 6, Lyon_district_merged.columns[[1] + list(range(5, Lyon_district_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,3e arrondissement,6,Grocery Store,Medical Center,Doctor's Office,Supermarket,Hospital,Food & Drink Shop,Medical Lab,Discount Store,Department Store,Theater


In [175]:
Lyon_district_merged.loc[Lyon_district_merged['Cluster Labels'] == 7, Lyon_district_merged.columns[[1] + list(range(5, Lyon_district_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


# Comments: 
Based-on analysis of different cluster the company RELOCATE ME found that district
9e arrondissement and 4e arrondissement have most visited Health center, Grocery store, Library along with social center, Performing Arts Venue. These two options could be the best match for their clients. Moreover, it also meets the client’s basic requirement, defined at the beginning of the problem.